---
title: Scientific project utils
---

## Inspirations

- [DrWatson.jl](https://juliadynamics.github.io/DrWatson.jl/stable/)
- [sacred](https://sacred.readthedocs.io/en/stable/)

In [ ]:
#| default_exp project
#| export
import os
import warnings
from typing import Any, Callable, Dict, List, Tuple, Union

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| exporti
def valtostring(val: Any, digits: Union[int, None], sigdigits: int) -> str:
    if isinstance(val, float):
        if digits is not None:
            return f"{val:.{digits}f}"
        elif sigdigits is not None:
            return f"{val:.{sigdigits}g}"
    return str(val)

In [ ]:
#| export
def savename(
    c: Dict,
    prefix: str = "",
    suffix: str = "",
    allowedtypes: Tuple = (int, float, str, bool),
    accesses: List[str] = [],
    ignores: List[str] = [],
    digits: Union[int, None] = None,
    connector: str = "_",
    expand: List[str] = [],
    sigdigits: int = 3,
    val2string: Callable = None,
    sort: bool = True,
    equals: str = "=",
) -> str:

    if any(sep in prefix for sep in ["/", "\\"]):
        warnings.warn(
            "Path separators in `savename` prefixes may break reproducibility on other OS. "
            "The recommended way is using `os.path.join` with `savename` (e.g. `os.path.join(datadir, savename(prefix, data))`)."
        )

    sigdigits = None if digits is not None else sigdigits
    val2string = val2string or (lambda val: valtostring(val, digits, sigdigits))

    labels = accesses or list(c.keys())
    p = (
        sorted(range(len(labels)), key=lambda k: labels[k])
        if sort
        else range(len(labels))
    )

    first = not prefix or prefix.endswith(os.path.sep)
    s = prefix
    for j in p:
        label = labels[j]
        if label in ignores:
            continue
        val = c[label]
        t = type(val)
        if any(issubclass(t, x) for x in allowedtypes):
            if label in expand:
                if not val:
                    continue
                sname = savename(
                    val,
                    connector=",",
                    digits=digits,
                    sigdigits=sigdigits,
                    equals=equals,
                )
                if not sname:
                    continue
                entry = f"{label}{equals}({sname})"
            else:
                entry = f"{label}{equals}{val2string(val)}"
            if not first:
                s += connector
            s += entry
            first = False

    if suffix:
        s += f".{suffix}"

    return s

In [ ]:
# Examples
d = {"a": 0.153456453, "b": 5, "mode": "double"}
test_eq(savename(d), "a=0.153_b=5_mode=double")
test_eq(savename(d, digits=4), "a=0.1535_b=5_mode=double")
test_eq(savename(d, prefix="n"), "n_a=0.153_b=5_mode=double")
test_eq(savename(d, suffix="n"), "a=0.153_b=5_mode=double.n")
test_eq(savename(d, prefix="n", suffix="n", connector="-"), "n-a=0.153-b=5-mode=double.n")
test_eq(savename(d, allowedtypes=(str,)), "mode=double")
test_eq(savename(d, connector=" | ", equals=" = "), "a = 0.153 | b = 5 | mode = double")

rick = {"never": "gonna", "give": "you", "up": "!"}
test_eq(savename(rick), "give=you_never=gonna_up=!")
test_eq(savename(rick, ignores=["up"]), "give=you_never=gonna")
